## 🔐 Prerequisites

Before running the first cell, make sure you're authenticated with Azure CLI:

```bash
az login
```

# 🏦 Suspend and Resume Conversation Threads

## Industry Use Case: Insurance Claim Processing Continuity

This notebook demonstrates how to **suspend and resume conversation threads** comparing service-managed (Azure AI) and in-memory threading approaches.

| Feature | FSI Application |
|---------|----------------|
| **Service-Managed Threads** | Azure-hosted claim conversations |
| **In-Memory Threads** | Local processing with custom storage |
| **Session Persistence** | Resume claims across sessions |
| **Multi-Device Access** | Continue claims from any device |

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv('../../.env', override=True)

print(f"✅ Environment loaded")
print(f"Project Endpoint: {os.getenv('AI_FOUNDRY_PROJECT_ENDPOINT')[:50]}...")

In [ ]:
from agent_framework.azure import AzureAIAgentClient, AzureOpenAIChatClient
from azure.identity.aio import AzureCliCredential

print("✅ All imports loaded")

## Define Insurance Claim Tools

In [ ]:
async def get_claim_status(claim_id: str) -> str:
    """Get insurance claim status."""
    claims = {
        "CLM-001": {"status": "Under Review", "type": "Auto", "amount": 5200.00, "filed": "2026-01-15"},
        "CLM-002": {"status": "Approved", "type": "Home", "amount": 15000.00, "filed": "2026-01-10"},
        "CLM-003": {"status": "Pending Documents", "type": "Health", "amount": 3500.00, "filed": "2026-01-17"},
    }
    c = claims.get(claim_id, {"status": "Not Found", "type": "Unknown", "amount": 0, "filed": "N/A"})
    return f"Claim {claim_id}: {c['type']} claim for ${c['amount']:,.2f}, Status: {c['status']}, Filed: {c['filed']}"


async def submit_additional_documents(claim_id: str, document_type: str) -> str:
    """Submit additional documents for a claim."""
    return f"Document '{document_type}' submission initiated for {claim_id}. Upload link will be sent to your registered email."


print("✅ Tools defined: get_claim_status, submit_additional_documents")

## Example 1: Service-Managed Thread (Azure AI)

Service-managed threads store conversation history in Azure. Serialization is lightweight (~50 bytes).

In [ ]:
async def suspend_resume_service_managed_thread():
    """Suspend and resume a service-managed thread (Azure AI)."""
    print("\n--- Suspend-Resume Service-Managed Thread (Azure AI) ---\n")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(credential=credential).as_agent(
            name="ClaimsAdvisor",
            instructions="You are an insurance claims advisor. Help customers with their claim inquiries.",
            tools=[get_claim_status, submit_additional_documents],
        ) as agent,
    ):
        # Start new thread
        thread = agent.get_new_thread()
        print("✅ Service-managed thread created\n")
        
        # Phase 1: Initial conversation
        query1 = "Hi, can you check the status of my claim CLM-003?"
        print(f"Customer: {query1}")
        response1 = await agent.run(query1, thread=thread)
        print(f"Advisor: {response1.text}\n")
        
        # Phase 2: Serialize (suspend)
        serialized_thread = await thread.serialize()
        print(f"💾 Serialized thread: {serialized_thread}")
        print(f"📊 Size: ~{len(str(serialized_thread))} bytes (just thread ID)\n")
        
        # Phase 3: Deserialize (resume)
        resumed_thread = await agent.deserialize_thread(serialized_thread)
        print("✅ Thread deserialized from Azure\n")
        
        query2 = "What documents do I need to provide?"
        print(f"Customer: {query2}")
        response2 = await agent.run(query2, thread=resumed_thread)
        print(f"Advisor: {response2.text}\n")
        
        print("✅ Service-managed thread demo completed!\n")

await suspend_resume_service_managed_thread()

## Example 2: In-Memory Thread (AzureOpenAIChatClient)

In-memory threads store full message history locally. Serialization includes all messages.

In [ ]:
async def suspend_resume_in_memory_thread():
    """Suspend and resume an in-memory thread (AzureOpenAIChatClient)."""
    print("\n--- Suspend-Resume In-Memory Thread (Azure OpenAI) ---\n")
    
    # Using AzureOpenAIChatClient for in-memory threading demo
    endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME", "gpt-4o"))
    
    agent = AzureOpenAIChatClient(
        endpoint=endpoint,
        deployment_name=deployment,
        credential=AzureCliCredential(),
    ).as_agent(
        name="ClaimsAdvisor",
        instructions="You are an insurance claims advisor. Help customers with their claim inquiries.",
        tools=[get_claim_status, submit_additional_documents],
    )
    
    # Start new thread
    thread = agent.get_new_thread()
    print("✅ In-memory thread created\n")
    
    # Phase 1: Initial conversation
    query1 = "Check status of claim CLM-001 please"
    print(f"Customer: {query1}")
    response1 = await agent.run(query1, thread=thread)
    print(f"Advisor: {response1.text}\n")
    
    # Phase 2: Serialize (suspend)
    serialized_thread = await thread.serialize()
    print(f"💾 Serialized thread (first 150 chars): {str(serialized_thread)[:150]}...")
    print(f"📊 Size: ~{len(str(serialized_thread))} bytes (full message history)\n")
    
    # Phase 3: Deserialize (resume)
    resumed_thread = await agent.deserialize_thread(serialized_thread)
    print("✅ Thread deserialized with full history\n")
    
    query2 = "What was the claim amount again?"
    print(f"Customer: {query2}")
    response2 = await agent.run(query2, thread=resumed_thread)
    print(f"Advisor: {response2.text}\n")
    
    print("✅ In-memory thread demo completed!\n")

await suspend_resume_in_memory_thread()

## APIs Used

| API | Description |
|-----|-------------|
| `AzureAIAgentClient.as_agent()` | Create Azure AI agent (service-managed) |
| `OpenAIChatClient().as_agent()` | Create OpenAI agent (in-memory) |
| `thread.serialize()` | Save thread state |
| `agent.deserialize_thread()` | Restore thread state |